In [1]:
import sqlalchemy
import sqlalchemy.ext.declarative
DB_NAME="sqlalchemy_levelup"

In [2]:
! mysql -e "DROP DATABASE IF EXISTS sqlalchemy_levelup"
! mysql -e "CREATE DATABASE IF NOT EXISTS sqlalchemy_levelup"

# What is SQLAlchemy?

From [their website](http://www.sqlalchemy.org/):

> SQLAlchemy is the Python SQL toolkit and Object Relational Mapper (ORM) that gives application developers the full power and flexibility of SQL

# What is the SQLAlchemy ORM?

The ORM provides a way of modelling the objects of your domain and the relationships between them in Python.

It will persist these objects and their relationships for you, using a Relational Database Management System (RDBMS)

# Where does the ORM fit into SQLAlchemy?

The ORM is built on top of SQLAlchemy's lower-level **Expression Language** API. This API is for working directly with database constructs and expressions in Python (e.g. it contains a `Select` class for building `select` queries)

It is possible, and sometimes preferable, to use ONLY the Expression Language if you require more granular control or if the overhead of the ORM is undesirable.

Alternatively, you can use a combination of both the ORM and the Expression Language to add more granular controlls to an ORM based application if and when they are needed

# A worked example - user management

Inspired by our Authentication service Thug Lyf

The domain objects are Users. Users will have attributes for their email address and password

# Defining the data model

As mentioned above, the ORM looks after the persistence of our domain objects.

Essentially this means we will create a Python class representing a `User`. Instances of this class will have attributes for email address and password. The ORM will store a representation of instantiated `User` objects as records in a "users" table in the database (e.g. MySQL, PostgreSQL).

The classes we add for our domain objects are referred to as *models* or *model classes*.

## Declarative Base Class

To have the ORM look after the persistence of our domain objects, we define our model classes as subclasses of a base class from SQLAlchemy.

This base class is provided by SQLAlchemy through the `declarative_base` class factory (method that returns a Python class)

It is common to have just one `Base` class in your application. You often see it defined in the module scope, allowing other modules to import it.

In [3]:
# Create a declarative base
Base = sqlalchemy.ext.declarative.declarative_base()

## The `User` Model

In [4]:
# Note the distinction between the singular "User" classname and the "users" table name
# This is because an instance of this class is a "user" but the table can contain many users
class User(Base):
    __tablename__ = "users"
    id = sqlalchemy.Column(sqlalchemy.Integer, primary_key=True)
    email = sqlalchemy.Column(sqlalchemy.String(255), nullable=False, unique=True)
    password = sqlalchemy.Column(sqlalchemy.String(255), nullable=False)
    
    def __repr__(self):
        return "User(email={})".format(self.email)

You might have noticed that the "type" of the columns here is either `Integer` or `String`, where elsewhere you may have seen specific `VARCHAR`, `CHAR`, `BIGINT` etc. The difference here is that `Integer` and `String` are generic "SQL" types, which will be represented as `VARCHAR`, `INT` etc. in the database. Using these generic types will make the application more independent from the choice of RDBMS.

### Table objects

When we declare a subclass of the declarative base (i.e. a model), the ORM will create a `Table` object and a mapping between the model attributes (id, email, password) and the `Table` columns.

`Table` is the SQLAlchemy abstraction representing RDBMS tables

In [5]:
# The table object associated with User is accessed by the __table__ class attribute
User.__table__

Table('users', MetaData(bind=None), Column('id', Integer(), table=<users>, primary_key=True, nullable=False), Column('email', String(length=255), table=<users>, nullable=False), Column('password', String(length=255), table=<users>, nullable=False), schema=None)

### Metadata

The declarative base has a `Metadata` object attached to it.

`Metadata` contains a registry of all the `Table` objects created when model classes are declared

We can access this registry under the `tables` attribute of the `metadata` object

In [6]:
Base.metadata.tables

immutabledict({'users': Table('users', MetaData(bind=None), Column('id', Integer(), table=<users>, primary_key=True, nullable=False), Column('email', String(length=255), table=<users>, nullable=False), Column('password', String(length=255), table=<users>, nullable=False), schema=None)})

The registry `Metadata` maintains will be useful in a minute when we want to create actual RDBMS tables

# Connecting to the database

The `Engine` is the source of database connectivity in SQLAlchemy. 

All SQLAlchemy applications will call upon an `Engine` instance at some point in order to create a `Connection` object. This is done through the ```engine.connect()``` method.

Note that this means that SQLAlchemy does not establish an actual connection with the database until it is requested 

In [7]:
database_url = sqlalchemy.engine.url.URL(drivername='mysql', database=DB_NAME,
                                         query={'read_default_file': '~/.my.cnf'})

In [8]:
database_url

mysql:///sqlalchemy_levelup?read_default_file=~/.my.cnf

In [9]:
engine = sqlalchemy.engine.create_engine(database_url, echo=True)  # echo=True logs SQL queries to stdout

## Creating tables

Now we have an engine to provide us with database connectivity, we can use the `Metadata` object to create database tables

In [10]:
Base.metadata.create_all(engine)

2017-01-05 13:52:33,730 INFO sqlalchemy.engine.base.Engine SHOW VARIABLES LIKE 'sql_mode'
2017-01-05 13:52:33,731 INFO sqlalchemy.engine.base.Engine ()
2017-01-05 13:52:33,733 INFO sqlalchemy.engine.base.Engine SELECT DATABASE()
2017-01-05 13:52:33,734 INFO sqlalchemy.engine.base.Engine ()
2017-01-05 13:52:33,735 INFO sqlalchemy.engine.base.Engine show collation where `Charset` = 'utf8' and `Collation` = 'utf8_bin'
2017-01-05 13:52:33,736 INFO sqlalchemy.engine.base.Engine ()
2017-01-05 13:52:33,738 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS CHAR(60)) AS anon_1
2017-01-05 13:52:33,739 INFO sqlalchemy.engine.base.Engine ()
2017-01-05 13:52:33,740 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS CHAR(60)) AS anon_1
2017-01-05 13:52:33,741 INFO sqlalchemy.engine.base.Engine ()
2017-01-05 13:52:33,742 INFO sqlalchemy.engine.base.Engine SELECT CAST('test collated returns' AS CHAR CHARACTER SET utf8) COLLATE utf8_bin AS anon_1
2017-01-05 13

# A Session with the ORM

Now that we have models, tables and an engine, we are ready to use the ORM to create and store `User` objects.

The ORM abstraction for managing synchronisation with the database is the **session**.

In [11]:
Session = sqlalchemy.orm.sessionmaker(bind=engine)

First, we create a `Session` class, configured to use the `Engine` object we created when it requires a database connection. `Session` objects will not ask for connections until they need them.

Now we can create a session:

In [12]:
session = Session()

And add our first user object

In [13]:
ben = User(email="ben@test.com", password="password")
print ben

session.add(ben)

User(email=ben@test.com)


The session can be thought of as a collection of objects. You can iterate over the session to access the objects within:

In [14]:
[obj for obj in session]

[User(email=ben@test.com)]

At this point, the user object exists in the session, but is not in the database (note that no SQL was emitted by the `echo`). To ask the ORM to flush changes to the database, we can use the `flush` method.

In [15]:
session.flush()

2017-01-05 13:52:33,803 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-01-05 13:52:33,806 INFO sqlalchemy.engine.base.Engine INSERT INTO users (email, password) VALUES (%s, %s)
2017-01-05 13:52:33,807 INFO sqlalchemy.engine.base.Engine ('ben@test.com', 'password')


`BEGIN` means a new transaction has been started on ORM connection with the database.

Right now the new record is in the database's transaction buffer (and therefore not visible to anyone else). To finish the transaction and fully persist the data, we can use `commit`

In [16]:
session.commit()

2017-01-05 13:52:33,814 INFO sqlalchemy.engine.base.Engine COMMIT


NB. We may also `commit` straightaway, without using `flush`

## Querying

Querying for data is achieved by using the `Session` object's `query` attribute

In [17]:
session.query(User).filter(User.email=="ben@example.com").all()

2017-01-05 13:52:33,822 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-01-05 13:52:33,824 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.email AS users_email, users.password AS users_password 
FROM users 
WHERE users.email = %s
2017-01-05 13:52:33,825 INFO sqlalchemy.engine.base.Engine ('ben@example.com',)


[]

Note that the `Session.query` method returns a `Query` object, which has methods such as `filter`, `order_by`, `limit` etc. These methods are *generative* - they return new `Query` objects allowing you to call additional methods. This is allows you to build up queries in long chains of method calls.

I won't go further into querying now, it's best to go and have a look at the [SQLAlchemy docs for the `Query` API](http://docs.sqlalchemy.org/en/latest/orm/query.html)

## Updating

If we wish to modify an object which has been placed in the session, we can just directly change the desired attribute's value. The ORM will detect the change and translate it into an `UPDATE` statement at the next `flush`

Let's say we want to improve Ben's password

In [18]:
ben.password = "drowssap"

In [19]:
session.commit()

2017-01-05 13:52:33,838 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.email AS users_email 
FROM users 
WHERE users.id = %s
2017-01-05 13:52:33,839 INFO sqlalchemy.engine.base.Engine (1L,)
2017-01-05 13:52:33,841 INFO sqlalchemy.engine.base.Engine UPDATE users SET password=%s WHERE users.id = %s
2017-01-05 13:52:33,842 INFO sqlalchemy.engine.base.Engine ('drowssap', 1L)
2017-01-05 13:52:33,844 INFO sqlalchemy.engine.base.Engine COMMIT


Any changes made to any objects in the session will be tracked by the ORM until a `flush` occurs.

Note that if the session is queried after an object has been modified, but before `flush` has been called, the ORM will issue a `flush` itself. This can bee seen in the example below, where the `UPDATE` statement happens before the `SELECT ... WHERE users.password` statement

In [20]:
ben.password = "new_password"

session.query(User).filter(User.password=="new_password").all()

2017-01-05 13:52:33,850 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-01-05 13:52:33,852 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.email AS users_email 
FROM users 
WHERE users.id = %s
2017-01-05 13:52:33,854 INFO sqlalchemy.engine.base.Engine (1L,)
2017-01-05 13:52:33,856 INFO sqlalchemy.engine.base.Engine UPDATE users SET password=%s WHERE users.id = %s
2017-01-05 13:52:33,857 INFO sqlalchemy.engine.base.Engine ('new_password', 1L)
2017-01-05 13:52:33,859 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.email AS users_email, users.password AS users_password 
FROM users 
WHERE users.password = %s
2017-01-05 13:52:33,859 INFO sqlalchemy.engine.base.Engine ('new_password',)


[User(email=ben@test.com)]

In [21]:
# Start fresh
session.close()  # ends the current session
Base.metadata.drop_all(bind=engine)  # drops all tables

2017-01-05 13:52:33,866 INFO sqlalchemy.engine.base.Engine ROLLBACK
2017-01-05 13:52:33,867 INFO sqlalchemy.engine.base.Engine DESCRIBE `users`
2017-01-05 13:52:33,868 INFO sqlalchemy.engine.base.Engine ()
2017-01-05 13:52:33,870 INFO sqlalchemy.engine.base.Engine 
DROP TABLE users
2017-01-05 13:52:33,871 INFO sqlalchemy.engine.base.Engine ()
2017-01-05 13:52:33,875 INFO sqlalchemy.engine.base.Engine COMMIT


# Extending the data model - Organisations

### Data model requirements

Our "customers" at GrowthIntel are businesses, not individuals. Because of this, our Auth service groups Users into larger units, which we call Organisations.

Organisations can have many Users. Users belong to one and only one Organisation (1 to many relationship)

### Diagram

```
      org1               org2
    /  |   \            /   \
user1 user2 user3    user4  user5
```

We can tell the ORM to handle relationships for us, with the `relationship` directive.

In order to create an relationship between `User` and `Organisation`, we need two things:

- A foreign key on `User` to specify the `Organisation` it belongs to
- An attribute we can use to refer to the related object(s) on each class

In [22]:
Base = sqlalchemy.ext.declarative.declarative_base()

class User(Base):
    __tablename__ = "users"
    id = sqlalchemy.Column(sqlalchemy.Integer, primary_key=True)
    email = sqlalchemy.Column(sqlalchemy.String(255), nullable=False, unique=True)
    password = sqlalchemy.Column(sqlalchemy.String(255), nullable=False)

    # New - foreign key to organisations table
    organisation_id = sqlalchemy.Column(
        sqlalchemy.Integer,
        sqlalchemy.ForeignKey('organisations.id'))

    # New - Define the ORM relationship
    organisation = sqlalchemy.orm.relationship("Organisation",
                                               back_populates="users")
    
    
    def __repr__(self):
        return "User(email={})".format(self.email)

class Organisation(Base):
    __tablename__ = "organisations"
    id = sqlalchemy.Column(sqlalchemy.Integer, primary_key=True)
    name = sqlalchemy.Column(sqlalchemy.String(255), nullable=False, unique=True)
    
    users = sqlalchemy.orm.relationship("User",
                                        back_populates="organisation")
    
    def __repr__(self):
        return "Organisation(name={})".format(self.name)

In [23]:
Base.metadata.create_all(bind=engine)

2017-01-05 13:52:33,903 INFO sqlalchemy.engine.base.Engine DESCRIBE `organisations`
2017-01-05 13:52:33,905 INFO sqlalchemy.engine.base.Engine ()
2017-01-05 13:52:33,906 INFO sqlalchemy.engine.base.Engine ROLLBACK
2017-01-05 13:52:33,907 INFO sqlalchemy.engine.base.Engine DESCRIBE `users`
2017-01-05 13:52:33,908 INFO sqlalchemy.engine.base.Engine ()
2017-01-05 13:52:33,909 INFO sqlalchemy.engine.base.Engine ROLLBACK
2017-01-05 13:52:33,910 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE organisations (
	id INTEGER NOT NULL AUTO_INCREMENT, 
	name VARCHAR(255) NOT NULL, 
	PRIMARY KEY (id), 
	UNIQUE (name)
)


2017-01-05 13:52:33,911 INFO sqlalchemy.engine.base.Engine ()
2017-01-05 13:52:33,932 INFO sqlalchemy.engine.base.Engine COMMIT
2017-01-05 13:52:33,933 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL AUTO_INCREMENT, 
	email VARCHAR(255) NOT NULL, 
	password VARCHAR(255) NOT NULL, 
	organisation_id INTEGER, 
	PRIMARY KEY (id), 
	UNIQUE (email), 
	FOREIG

Now I can create Ben in an organisation Example.com, and give him some colleagues

In [24]:
example_com = Organisation(name="example.com")

ben = User(email="ben@example.com",
           password="password1",
           organisation=example_com)

jerry = User(email="jerry@example.com",
             password="password2",
             organisation=example_com)

ellie = User(email="ellie@example.com",
             password="password3",
             organisation=example_com)

When I now add these objects to a session and commit, the ORM will handle the "details" of foreign keys for me 

In [25]:
session = Session()

session.add_all([example_com, ben, jerry, ellie])

session.commit()

2017-01-05 13:52:33,970 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-01-05 13:52:33,972 INFO sqlalchemy.engine.base.Engine INSERT INTO organisations (name) VALUES (%s)
2017-01-05 13:52:33,973 INFO sqlalchemy.engine.base.Engine ('example.com',)
2017-01-05 13:52:33,975 INFO sqlalchemy.engine.base.Engine INSERT INTO users (email, password, organisation_id) VALUES (%s, %s, %s)
2017-01-05 13:52:33,976 INFO sqlalchemy.engine.base.Engine ('ben@example.com', 'password1', 1L)
2017-01-05 13:52:33,977 INFO sqlalchemy.engine.base.Engine INSERT INTO users (email, password, organisation_id) VALUES (%s, %s, %s)
2017-01-05 13:52:33,978 INFO sqlalchemy.engine.base.Engine ('jerry@example.com', 'password2', 1L)
2017-01-05 13:52:33,979 INFO sqlalchemy.engine.base.Engine INSERT INTO users (email, password, organisation_id) VALUES (%s, %s, %s)
2017-01-05 13:52:33,980 INFO sqlalchemy.engine.base.Engine ('ellie@example.com', 'password3', 1L)
2017-01-05 13:52:33,981 INFO sqlalchemy.engine.base.Engi

Using the ORM relationships allow us to play around with the relationships between objects in a way that would normally require a lot of faffy SQL `UPDATE`, `SELECT .. JOIN` statements

For example, we can treat the `example_com.users` attribute as an ordinary Python collection or objects

In [26]:
example_com.users

2017-01-05 13:52:33,987 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-01-05 13:52:33,988 INFO sqlalchemy.engine.base.Engine SELECT organisations.id AS organisations_id, organisations.name AS organisations_name 
FROM organisations 
WHERE organisations.id = %s
2017-01-05 13:52:33,989 INFO sqlalchemy.engine.base.Engine (1L,)
2017-01-05 13:52:33,991 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.email AS users_email, users.password AS users_password, users.organisation_id AS users_organisation_id 
FROM users 
WHERE %s = users.organisation_id
2017-01-05 13:52:33,992 INFO sqlalchemy.engine.base.Engine (1L,)


[User(email=ben@example.com),
 User(email=jerry@example.com),
 User(email=ellie@example.com)]

In [27]:
example_com.users.remove(ben)

In [28]:
rosie = User(email="rosie@example.com", password="password4")

example_com.users.append(rosie)

In [29]:
session.flush()

2017-01-05 13:52:34,011 INFO sqlalchemy.engine.base.Engine UPDATE users SET organisation_id=%s WHERE users.id = %s
2017-01-05 13:52:34,013 INFO sqlalchemy.engine.base.Engine (None, 1L)
2017-01-05 13:52:34,015 INFO sqlalchemy.engine.base.Engine INSERT INTO users (email, password, organisation_id) VALUES (%s, %s, %s)
2017-01-05 13:52:34,016 INFO sqlalchemy.engine.base.Engine ('rosie@example.com', 'password4', 1L)


In [30]:
example_com.users

[User(email=jerry@example.com),
 User(email=ellie@example.com),
 User(email=rosie@example.com)]

In [31]:
rosie.organisation

Organisation(name=example.com)

In [32]:
session.close()
Base.metadata.drop_all(bind=engine)

2017-01-05 13:52:34,036 INFO sqlalchemy.engine.base.Engine ROLLBACK
2017-01-05 13:52:34,038 INFO sqlalchemy.engine.base.Engine DESCRIBE `organisations`
2017-01-05 13:52:34,040 INFO sqlalchemy.engine.base.Engine ()
2017-01-05 13:52:34,041 INFO sqlalchemy.engine.base.Engine DESCRIBE `users`
2017-01-05 13:52:34,042 INFO sqlalchemy.engine.base.Engine ()
2017-01-05 13:52:34,043 INFO sqlalchemy.engine.base.Engine 
DROP TABLE users
2017-01-05 13:52:34,044 INFO sqlalchemy.engine.base.Engine ()
2017-01-05 13:52:34,047 INFO sqlalchemy.engine.base.Engine COMMIT
2017-01-05 13:52:34,048 INFO sqlalchemy.engine.base.Engine 
DROP TABLE organisations
2017-01-05 13:52:34,049 INFO sqlalchemy.engine.base.Engine ()
2017-01-05 13:52:34,051 INFO sqlalchemy.engine.base.Engine COMMIT


# Many-to-Many relationships - User Roles

### Data model requirements

User's have permissions known as roles. There are a set number of roles, and any user can have 0, 1 or many of them.

### Diagram

TODO

This implies a 1-to-many relationship, which is not quite as straightforward to set up with the ORM, but just as easy to use.

The complicating factor is that we can't use simple foreign key relationships, as in effect we need many foreign keys per `User`.

This is commonly solved in a relational database using a third *association table* which contains pairs of foreign keys `user_id`-`role_id`

In [34]:
Base = sqlalchemy.ext.declarative.declarative_base()

# NEW - association table
user_roles = sqlalchemy.Table(
    'user_roles', Base.metadata,
    sqlalchemy.Column('user_id', sqlalchemy.ForeignKey('users.id'), primary_key=True),
    sqlalchemy.Column('role_id', sqlalchemy.ForeignKey('roles.id'), primary_key=True),
)

class User(Base):
    __tablename__ = "users"
    id = sqlalchemy.Column(sqlalchemy.Integer, primary_key=True)
    email = sqlalchemy.Column(sqlalchemy.String(255), nullable=False, unique=True)
    password = sqlalchemy.Column(sqlalchemy.String(255), nullable=False)

    # NEW - relationship to Role via secondary table
    roles = sqlalchemy.orm.relationship("Role",
                                        secondary=user_roles,
                                        back_populates="users")
    
    def __repr__(self):
        return "User(email={})".format(self.email)
    
class Role(Base):
    __tablename__ = "roles"
    id = sqlalchemy.Column(sqlalchemy.Integer, primary_key=True)
    name = sqlalchemy.Column(sqlalchemy.String(255), nullable=False, unique=True)
    
    users = sqlalchemy.orm.relationship("User",
                                        secondary=user_roles,
                                        back_populates="roles")

There's a couple of things to note.

We didnt use a model class for the association table, instead directly using the `Table` object. This is because a 'user_role' is not a domain object to be created and modified in our application. Its more of a second class citizen which only the ORM needs to know about.

Second, we give the new `Table` access to the `Metadata` object, so it is tracked in its registry and all `create_all`, `drop_all` commands will include this table.

In [35]:
Base.metadata.create_all(bind=engine)

2017-01-05 13:53:05,494 INFO sqlalchemy.engine.base.Engine DESCRIBE `user_roles`
2017-01-05 13:53:05,496 INFO sqlalchemy.engine.base.Engine ()
2017-01-05 13:53:05,497 INFO sqlalchemy.engine.base.Engine ROLLBACK
2017-01-05 13:53:05,498 INFO sqlalchemy.engine.base.Engine DESCRIBE `roles`
2017-01-05 13:53:05,499 INFO sqlalchemy.engine.base.Engine ()
2017-01-05 13:53:05,500 INFO sqlalchemy.engine.base.Engine ROLLBACK
2017-01-05 13:53:05,501 INFO sqlalchemy.engine.base.Engine DESCRIBE `users`
2017-01-05 13:53:05,501 INFO sqlalchemy.engine.base.Engine ()
2017-01-05 13:53:05,503 INFO sqlalchemy.engine.base.Engine ROLLBACK
2017-01-05 13:53:05,505 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE roles (
	id INTEGER NOT NULL AUTO_INCREMENT, 
	name VARCHAR(255) NOT NULL, 
	PRIMARY KEY (id), 
	UNIQUE (name)
)


2017-01-05 13:53:05,506 INFO sqlalchemy.engine.base.Engine ()
2017-01-05 13:53:05,522 INFO sqlalchemy.engine.base.Engine COMMIT
2017-01-05 13:53:05,524 INFO sqlalchemy.engine.base.Engine 
C

Let's create a set of `Role` and `User` objects to play around with

In [36]:
ben = User(email="ben@example.com",
           password="password1")

jerry = User(email="jerry@example.com",
             password="password2")

ellie = User(email="ellie@example.com",
             password="password3")

In [37]:
admin = Role(name="admin")
test_user = Role(name="test_user")
can_export = Role(name="can_export")

In [38]:
ben.roles.extend([admin, can_export])
jerry.roles.append(test_user)

In [39]:
session.add_all([ben, jerry, ellie])

In [40]:
session.commit()

2017-01-05 13:59:48,646 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-01-05 13:59:48,649 INFO sqlalchemy.engine.base.Engine INSERT INTO roles (name) VALUES (%s)
2017-01-05 13:59:48,650 INFO sqlalchemy.engine.base.Engine ('admin',)
2017-01-05 13:59:48,651 INFO sqlalchemy.engine.base.Engine INSERT INTO roles (name) VALUES (%s)
2017-01-05 13:59:48,652 INFO sqlalchemy.engine.base.Engine ('can_export',)
2017-01-05 13:59:48,653 INFO sqlalchemy.engine.base.Engine INSERT INTO roles (name) VALUES (%s)
2017-01-05 13:59:48,654 INFO sqlalchemy.engine.base.Engine ('test_user',)
2017-01-05 13:59:48,655 INFO sqlalchemy.engine.base.Engine INSERT INTO users (email, password) VALUES (%s, %s)
2017-01-05 13:59:48,656 INFO sqlalchemy.engine.base.Engine ('ben@example.com', 'password1')
2017-01-05 13:59:48,658 INFO sqlalchemy.engine.base.Engine INSERT INTO users (email, password) VALUES (%s, %s)
2017-01-05 13:59:48,658 INFO sqlalchemy.engine.base.Engine ('jerry@example.com', 'password2')
2017-01-0

# Testing Applications with SQLAlchemy

Transactions and rollbacks